In [ ]:
graph_out = { }
with open('web-Google.txt','r') as file:
    for line in file:
        if line[0]=='#':
            continue
        val = line.split()
        k=int(val[0])
        v=int(val[1])
        if k not in graph_out.keys():
            graph_out.update({k:[v]})
        else:
            graph_out[k].append(v)
        if v not in graph_out.keys():
            graph_out.update({v:[]})          

In [ ]:
graph_in = { }
with open('web-Google.txt','r') as file:
    for line in file:
        if line[0]=='#':
            continue
        val = line.split()
        k=int(val[1])
        v=int(val[0])
        if k not in graph_in.keys():
            graph_in.update({k:[v]})
        else:
            graph_in[k].append(v)
        if v not in graph_in.keys():
            graph_in.update({v:[]})          

In [ ]:
#initializing page ranks
import copy
import time
def init_rank():
    global graph_in
    p={}
    for key in graph_in.keys():
        p.update({key:1})
    return p
#note that node ids are not continuous

In [ ]:
def rank(pp,n):   #remember that p is a mutable object
    global graph_in
    global graph_out
    p=copy.deepcopy(pp)
    for i in range(n):
        p_old = p
        p = {}
        for keyj in graph_in.keys():
            summ=0
            for keyi in graph_in[keyj]:
                summ+=(1/len(graph_out[keyi])) * p_old[keyi]
            p.update({keyj:summ})
    return p

In [ ]:
def pranking(n):   
    pp=init_rank()
    page_rank = rank(pp,n)  #running for n iterations
    pr_sorted = sorted(page_rank.items(),key=lambda x:x[1],reverse=True)
    print('Here are the top 20 ranked pages')
    print('(Node id, Page rank)')
    for v in pr_sorted[:20]:
        print(v)
    print('\nHere are the bottom 20  ranked pages')
    print('(Node id, Page rank)')
    for v1 in pr_sorted[-20:]:
        print(v1)#print here
    return pr_sorted

In [ ]:
pr=pranking(10)

In [ ]:
def rank2(pp,n,a):   #remember that p is a mutable object, question 2 starts here
    global graph_in
    global graph_out
    p=copy.deepcopy(pp)
    for i in range(n):
        p_old = p
        p = {}
        for keyj in graph_in.keys():
            summ=0
            for keyi in graph_in[keyj]:
                summ+=(1/len(graph_out[keyi])) * p_old[keyi]
            p.update({keyj:a*summ+(1-a)})
    return p

In [ ]:
def pranking2(n,a):
    pp = init_rank()
    page_rank2=rank2(pp,n,a)
    pr_sorted2 = sorted(page_rank2.items(),key=lambda x:x[1],reverse=True)
    print('Here are the top 20 ranked pages')
    print('(Node id, Page rank)')
    for v in pr_sorted2[:20]:
        print(v)
    print('\nHere are the bottom 20  ranked pages')
    print('(Node id, Page rank)')
    for v1 in pr_sorted2[-20:]:
        print(v1)#print here
    return pr_sorted2

In [ ]:
p_rank02_10 = pranking2(10,0.2)

In [ ]:
p_rank85_10 = pranking2(10,0.85)

In [ ]:
import matplotlib.pyplot as plt
import math
from matplotlib.ticker import FixedLocator as fxd
def plotxy(sorted_list,s):
    x2=[x[1] for x in sorted_list]
    count = [0 for i in range(200)]
    stepy=[0]
    step = (max(x2) - min(x2))/200
    x = [step/2]
    for val in range(199):
        x.append(x[-1]+step)
        stepy.append(stepy[-1]+step)
    stepy.append(stepy[-1]+step)
    for i in range(200):
        for val in x2:
            if stepy[i]<val and val<stepy[i+1]:
                count[i]+=1
    xvalues = []
    yvalues = []
    min_ticks=[]
    for i in range(200):
        if count[i]>0:
            xvalues.append(math.log10(x[i]))
            yvalues.append(math.log10(count[i]))
            min_ticks.append(math.log10(stepy[i+1]))
    fig, ax = plt.subplots()
    ax.scatter(xvalues,yvalues,marker='+')
    minor_locator = fxd(min_ticks)
    ax.xaxis.set_minor_locator(minor_locator)
    ax.set_ylim(bottom=0)      
    #plt.scatter(xvalues,yvalues)
    plt.xlabel('page rank(log10)')
    plt.ylabel('counts(log10)')
    plt.title(s)
    plt.show()
    #make plots with zero dissapear since log is not possible

In [ ]:
plotxy(p_rank02_10,'Page rank with a=0.2 and 10 iterations')

In [ ]:
plotxy(p_rank85_10,'Page rank with a=0.85 and 10 iterations')

Observations from the graphs above : There are more low ranked pages than there are high ranked pages

Question c starts from here

In [ ]:
#section a
timings_a=[]
for n in (50,100,200):
    st_time=time.time()
    print('\n For iteration number = {}'.format(n))
    pr1=pranking(n)
    timings_a.append(time.time()-st_time)

In [ ]:
timings_a

In [ ]:
#section b
def repeat_anb(a):
    timings_b=[]
    for n in (50,100,200):
        print('\n For iteration number = {}'.format(n))
        st_time=time.time()
        pr2=pranking2(n,a)
        plotxy(pr2,'Page rank with a={} and number of iterations={}'.format(a,n))
        timings_b.append(time.time() - st_time)
    return timings_b

In [ ]:
time02=repeat_anb(0.2)

In [ ]:
time02

In [ ]:
time85=repeat_anb(0.85)

In [ ]:
time85

In [ ]:
#stop when p changes no further..can be implemented in rank by comparing p in each iteration
#sorting in pranking has to go up to ranking, so that the sorted list has to be arrange to ensure stop
#introduced timing to repeat

section d starts here: How to detect convergence earlier

In [ ]:
def rank_conv(pp,n):   #remember that p is a mutable object
    global graph_in
    global graph_out
    p=copy.deepcopy(pp)
    for i in range(n):
        p_old = p
        p = {}
        for keyj in graph_in.keys():
            summ=0
            for keyi in graph_in[keyj]:
                summ+=(1/len(graph_out[keyi])) * p_old[keyi]
            p.update({keyj:summ})
        old_rank=[k[1] for k in p_old.items()]
        new_rank=[k[1] for k in p.items()]
        err = sum([abs(x-y) for x,y in zip(old_rank,new_rank)]) *(1/len(old_rank))
        #print(err)
        if err <= 2e-2:
            print('Converges at iteration number {}'.format(i))
            return p
    return p
def pranking_conv(n):   
    pp=init_rank()
    page_rank = rank_conv(pp,n)  #running for n iterations
    pr_sorted = sorted(page_rank.items(),key=lambda x:x[1],reverse=True)
    print('Here are the top 20 ranked pages')
    print('(Node id, Page rank)')
    for v in pr_sorted[:20]:
        print(v)
    print('\nHere are the bottom 20  ranked pages')
    print('(Node id, Page rank)')
    for v1 in pr_sorted[-20:]:
        print(v1)#print here
    return

In [ ]:
def convergence_check(pp,n,a):
    global graph_in
    global graph_out
    p=copy.deepcopy(pp)
    arr=[]
    for i in range(n):
        p_old = p
        p = {}
        for keyj in graph_in.keys():
            summ=0
            for keyi in graph_in[keyj]:
                summ+=(1/len(graph_out[keyi])) * p_old[keyi]
            p.update({keyj:a*summ+(1-a)})
        old_rank=[k[1] for k in p_old.items()]
        new_rank=[k[1] for k in p.items()]
        arr.append(sum([abs(x-y) for x,y in zip(old_rank,new_rank)]) * (1/len(new_rank)))
    return arr

In [ ]:
bb=convergence_check(init_rank(),50,1)
bb_02=convergence_check(init_rank(),50,0.2)
bb_85=convergence_check(init_rank(),50,0.85)
plt.plot(list(range(1,51)),bb)
plt.plot(list(range(1,51)),bb_02)
plt.plot(list(range(1,51)),bb_85)
plt.legend(['simple version','damping factor a=0.2','damping factor a=0.85'])
plt.xlabel('iterations')
plt.ylabel('mean absolute error')
plt.title('Error vs Iteration')
plt.show()

In [ ]:
pranking_conv(200)

In [ ]:
def rank2_conv(pp,n,a):   
    global graph_in
    global graph_out
    p=copy.deepcopy(pp)
    for i in range(n):
        p_old = p
        p = {}
        for keyj in graph_in.keys():
            summ=0
            for keyi in graph_in[keyj]:
                summ+=(1/len(graph_out[keyi])) * p_old[keyi]
            p.update({keyj:a*summ+(1-a)})
        old_rank=[k[1] for k in p_old.items()]
        new_rank=[k[1] for k in p.items()]
        if sum([abs(x-y) for x,y in zip(old_rank,new_rank)]) * (1/len(new_rank)) <= 1e-3:
            print('Converges at iteration number {}'.format(i))
            return p
    return p

def pranking2_conv(n,a):
    pp = init_rank()
    page_rank2=rank2_conv(pp,n,a)
    pr_sorted2 = sorted(page_rank2.items(),key=lambda x:x[1],reverse=True)
    print('Here are the top 20 ranked pages')
    print('(Node id, Page rank)')
    for v in pr_sorted2[:20]:
        print(v)
    print('\nHere are the bottom 20  ranked pages')
    print('(Node id, Page rank)')
    for v1 in pr_sorted2[-20:]:
        print(v1)#print here
    return pr_sorted2

In [ ]:
aa=pranking2_conv(200,0.2)

In [ ]:
aa=pranking2_conv(200,0.85)